Topologies are modeled in terms of the relative bandwidths of the links. In this version of the problem, we assume that all per-send latencies are uniform, which is mostly true over NVLinks.

In [1]:
from msccl.topologies import dgx1
from pprint import pprint
topology = dgx1()
pprint(topology.links)

[[0, 2, 1, 1, 2, 0, 0, 0],
 [2, 0, 1, 2, 0, 1, 0, 0],
 [1, 1, 0, 2, 0, 0, 2, 0],
 [1, 2, 2, 0, 0, 0, 0, 1],
 [2, 0, 0, 0, 0, 2, 1, 1],
 [0, 1, 0, 0, 2, 0, 1, 2],
 [0, 0, 2, 0, 1, 1, 0, 2],
 [0, 0, 0, 1, 1, 2, 2, 0]]


The collective is the specification for where chunks start at and where they need to go. Here we instantiate allgather for this topology.

In [2]:
from msccl.collectives import allgather
collective = allgather(topology.num_nodes())

Here is the precondition. We can see that all ranks start with one chunk.

In [3]:
pprint([[1 if collective.precondition(rank, chunk) else 0 for chunk in range(collective.num_chunks)] for rank in range(collective.num_nodes)])

[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1]]


Here is the postcondition. All ranks need to get all chunks.

In [4]:
pprint([[1 if collective.postcondition(rank, chunk) else 0 for chunk in range(collective.num_chunks)] for rank in range(collective.num_nodes)])

[[1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]


Lets try to actually solve this for a specific number of steps. `msccl.strategies` offers entry points into the solver. We'll use one that just does a single solver call for now. The encoding itself lives in [path_encoding.py](../msccl/path_encoding.py). As expected, 1 step is not enough, because some ranks aren't directly connected.

In [5]:
from msccl.strategies import solve_instance
from msccl.instance import Instance
algo = solve_instance(topology, collective, Instance(steps=1), logging=True)

Solving instance steps=1... unsatisfiable. (0.2s)


But 2 steps is.

In [6]:
algo = solve_instance(topology, collective, Instance(steps=2), logging=True)

Solving instance steps=2... synthesized! (0.3s)


The algorithm is composed of the sends to perform in each global step in `(chunk, source, destination)` form. The `rounds` is how many multiples of the topology's available bandwidth is needed for that step.

In [7]:
pprint(algo.steps)

[Step(rounds=1, sends=[(0, 0, 1), (0, 0, 3), (0, 0, 4), (1, 1, 2), (1, 1, 3), (1, 1, 5), (2, 2, 1), (2, 2, 3), (2, 2, 6), (3, 3, 1), (3, 3, 2), (3, 3, 7), (4, 4, 0), (4, 4, 6), (4, 4, 7), (5, 5, 1), (5, 5, 7), (6, 6, 4), (6, 6, 5), (6, 6, 7), (7, 7, 3), (7, 7, 4)]),
 Step(rounds=1, sends=[(0, 3, 2), (0, 4, 5), (0, 4, 6), (0, 4, 7), (1, 2, 0), (1, 2, 6), (1, 3, 7), (1, 5, 4), (2, 1, 0), (2, 6, 4), (2, 6, 5), (2, 6, 7), (3, 1, 5), (3, 2, 6), (3, 3, 0), (3, 7, 4), (4, 0, 1), (4, 0, 2), (4, 0, 3), (4, 7, 5), (5, 1, 0), (5, 1, 2), (5, 1, 3), (5, 5, 4), (5, 5, 6), (6, 4, 0), (6, 5, 1), (6, 6, 2), (6, 7, 3), (7, 3, 1), (7, 3, 2), (7, 4, 0), (7, 7, 5), (7, 7, 6)])]


Neither of those instances considered dividing the chunks into smaller ones for more fine grained routing. That can be achieved by passing `chunks=N` to the `Instance`. The bandwidths in the topology are stated relative to the chunk size, so when the chunks parameter goes up, more steps may be needed. For example:

In [8]:
algo = solve_instance(topology, collective, Instance(steps=2, chunks=2), logging=True)
algo = solve_instance(topology, collective, Instance(steps=3, chunks=2), logging=True)

Solving instance steps=2,chunks=2... unsatisfiable. (0.4s)
Solving instance steps=3,chunks=2... synthesized! (0.7s)


However, it turns out that 2 steps is enough *if we allow one step to take double the time*. The solver can be give these "extra rounds" of bandwidth to allocate to the steps with an `extra_rounds` parameter:

In [9]:
algo = solve_instance(topology, collective, Instance(steps=2, chunks=2, extra_rounds=1), logging=True)

Solving instance steps=2,rounds=3,chunks=2... synthesized! (0.7s)


In an alpha+beta cost model, `steps` is essentially how many times the alpha cost is paid, while the multiple for beta is `size*rounds/chunks`, where `size` is the size of the input. We've automated searching over different tradeoffs between steps, rounds and chunks in a `solve_all_latency_bandwidth_tradeoffs` strategy:

In [10]:
from msccl.strategies import solve_all_latency_bandwidth_tradeoffs
algos = list(solve_all_latency_bandwidth_tradeoffs(topology, collective, logging=True))

Algorithms need at least 2 steps.
Algorithms need at least 7/6 rounds per chunk.
Solving instance steps=2... synthesized! (0.3s)
Solving instance steps=2,rounds=3,chunks=2... synthesized! (0.6s)
Solving instance steps=2,rounds=4,chunks=3... unsatisfiable. (0.8s)
Solving instance steps=3,rounds=4,chunks=3... synthesized! (1.5s)
Solving instance steps=2,rounds=5,chunks=4... unsatisfiable. (1.3s)
Solving instance steps=3,rounds=5,chunks=4... synthesized! (6.8s)
Solving instance steps=2,rounds=6,chunks=5... unsatisfiable. (1.8s)
Solving instance steps=3,rounds=6,chunks=5... synthesized! (13.1s)
Solving instance steps=2,rounds=7,chunks=6... unsatisfiable. (2.9s)
Solving instance steps=3,rounds=7,chunks=6... synthesized! (124.1s)
Bandwidth optimal algorithm found!


Two preprocessing steps are performed:
- The minimum number of steps required is lower bound based on the maximum of the shortest paths for each chunk considering the topology.
- A minimum number of rounds per chunk is lower bound using a kind of multi-commodity flow encoding in [rounds_bound.py](../msccl/rounds_bound.py).

Then all relevant trade-offs are iterated until a bandwidth optimal algorithm is found (if the rounds per chunk lower bound happens to be exact).

The synthesized algorithms contain many non-Pareto-optimal algorithms, which are dominated by some other algorithm for all input sizes. We can filter those out:

In [11]:
from msccl.strategies import prune_pareto_optimal
algos = prune_pareto_optimal(algos)

Lets set up a function to analyze the performance of the remaining algorithms. Here we assume that alpha=1 and beta=1.

In [13]:
from fractions import Fraction
def print_perf(size):
    print(f'Input size is {size}')
    for algo in algos:
        print(f'\n{algo.name}')
        chunk_size = Fraction(1, algo.instance.chunks)
        print(f'Chunk size:             1/chunks = {chunk_size} = {float(chunk_size)}')
        bw_mult = algo.instance.rounds() * chunk_size
        print(f'BW multiples:      rounds/chunks = {bw_mult} = {float(bw_mult)}')
        time = algo.instance.steps + size * bw_mult
        print(f'Time: steps + size*rounds/chunks = {time}')

When the input size is large, the second algorithm is better:

In [14]:
print_perf(10.0)

Input size is 10.0

Allgather(n=8)-DGX1-steps=2,rounds=3,chunks=2
Chunk size:             1/chunks = 1/2 = 0.5
BW multiples:      rounds/chunks = 3/2 = 1.5
Time: steps + size*rounds/chunks = 17.0

Allgather(n=8)-DGX1-steps=3,rounds=7,chunks=6
Chunk size:             1/chunks = 1/6 = 0.16666666666666666
BW multiples:      rounds/chunks = 7/6 = 1.1666666666666667
Time: steps + size*rounds/chunks = 14.666666666666668


For small inputs the first one is:

In [15]:
print_perf(0.1)

Input size is 0.1

Allgather(n=8)-DGX1-steps=2,rounds=3,chunks=2
Chunk size:             1/chunks = 1/2 = 0.5
BW multiples:      rounds/chunks = 3/2 = 1.5
Time: steps + size*rounds/chunks = 2.15

Allgather(n=8)-DGX1-steps=3,rounds=7,chunks=6
Chunk size:             1/chunks = 1/6 = 0.16666666666666666
BW multiples:      rounds/chunks = 7/6 = 1.1666666666666667
Time: steps + size*rounds/chunks = 3.1166666666666667
